The [Phoible](https://phoible.org/) dataset contains phoneme inventories for thousands of languages and dialects. Many languages/dialicts have multiple Phoible records. Here, I'm mapping the data against pre-prepared IPA phoneme tables, then selecting one sample per table per language so that each language is only represented once in the final dataset (to avoid bias by oversampling).

The IPA phoneme tables contain some phonemes with diacritics and some without. They seem to include only 0 or 1 diacritic perphoneme. 

The Phoible data also contains phonemes with and without diacritics, but they may contain sequences of multiple diacritics after each phoneme. These need to be split and handled seperately.



In [128]:
from pathlib import Path
import requests
from tqdm import tqdm
import matplotlib.pyplot as plt

%matplotlib inline
import pandas as pd
import numpy as np
from IPython.display import display


phoible_data_url = "https://raw.githubusercontent.com/phoible/dev/v2.0/data/phoible.csv"
phobile_file_path = "./downloads/phoible.csv"

VALIDATE_RESULTS = True

if not Path(phobile_file_path).exists():
    response = requests.get(phoible_data_url, stream=True)

    with open(phobile_file_path, "wb") as fh:
        for data in tqdm(response.iter_content()):
            fh.write(data)

In [129]:
ipa_diacritics_df = pd.read_csv(
    "./data/ipa_diacritics.csv", dtype=str, encoding="utf-8", engine="python"
)
valid_diacritics = ipa_diacritics_df.suffix
print(ipa_diacritics_df.shape)
ipa_diacritics_df.head(3)

(38, 2)


,suffix,description
0,NaN,Standard
1,ː,Long
2,ˑ,Half-long


Load the Phoible data and split the IPA diacritic suffixes from all phonemes so the raw phoneme can be matched against the pre-prepared IPA phoneme tables.

In [130]:
phoible_df = pd.read_csv(phobile_file_path, encoding="utf-8", engine="python")
dialect_phonemes_df = (
    phoible_df[["ISO6393", "LanguageName", "SpecificDialect", "Phoneme"]]
    .copy()
    .fillna({"SpecificDialect": "none"})
)


def split_symbol(phoneme):
    return "".join(
        [x for x in list(phoneme) if x != "" and x not in valid_diacritics.values]
    )


def split_diacritics(phoneme):
    return "".join(
        [x for x in list(phoneme) if x != "" and x in valid_diacritics.values]
    )


def split_phoneme_series(df):
    df["symbol"] = df.Phoneme.apply(split_symbol)
    df["diacritics"] = df.Phoneme.apply(split_diacritics)
    df["diacritic_count"] = df.diacritics.str.len()
    return df


dialect_phonemes_df = split_phoneme_series(dialect_phonemes_df)

display(dialect_phonemes_df.shape)
dialect_phonemes_df.head(3)

(105467, 7)

,ISO6393,LanguageName,SpecificDialect,Phoneme,symbol,diacritics,diacritic_count
0,kor,Korean,none,a,a,,0
1,kor,Korean,none,aː,a,ː,1
2,kor,Korean,none,æ,æ,,0


In [131]:
lang_by_dialect_df = dialect_phonemes_df.groupby(
    ["LanguageName", "SpecificDialect"]
)

language_names = (
    lang_by_dialect_df.first()
    .index.get_level_values(0)
    .to_series()
    .reset_index(drop=True)
)

num_lang = len(language_names)

print("num_lang", num_lang)

num_lang 2949


Load the pre-prepared IPA tables.

In [132]:
cons_pl_tbl_df = pd.read_csv(
    "./data/consonants_plumonic.csv",
    dtype=str,
    index_col=[0],
    keep_default_na=False,
    na_values=["-1"],
)
display(cons_pl_tbl_df)

cons_npl_tbl_df = pd.read_csv(
    "./data/consonants_non_plumonic.csv",
    dtype=str,
    index_col=[0],
    keep_default_na=False,
    na_values=["-1"],
)
display(cons_npl_tbl_df)

cons_coart_tbl_df = pd.read_csv(
    "./data/consonants_coarticulated.csv",
    dtype=str,
    index_col=[0],
    keep_default_na=False,
    na_values=["-1"],
)
display(cons_coart_tbl_df)

vowels_tbl_df = pd.read_csv(
    "./data/vowels.csv",
    dtype=str,
    index_col=[0],
    keep_default_na=False,
    na_values=["-1"],
)
display(vowels_tbl_df)


cons_pl_tbl_shape = [*cons_pl_tbl_df.shape]
cons_npl_tbl_shape = [*cons_npl_tbl_df.shape]
cons_coart_tbl_shape = [*cons_coart_tbl_df.shape]
vowels_tbl_shape = [*vowels_tbl_df.shape]

cons_pl_tbl_shape, cons_npl_tbl_shape, cons_coart_tbl_shape, vowels_tbl_shape

,Bilabial,Bilabial_v,Labiodental,Labiodental_v,Dental,Dental_v,Alveolar,Alveolar_v,Postalveolar,Postalveolar_v,...,Palatal,Palatal_v,Velar,Velar_v,Uvular,Uvular_v,Pharyngeal,Pharyngeal_v,Glottal,Glottal_v
Plosive,p,b,,,t̪,d̪,t,d,,,...,c,ɟ,k,ɡ,q,ɢ,,NaN,ʔ,NaN
Nasal,,m,,ɱ,,n̪,,n,,,...,,ɲ,,ŋ,,ɴ,NaN,NaN,NaN,NaN
Trill,,ʙ,,,,r̪,,r,,,...,,,NaN,NaN,,ʀ,,,NaN,NaN
Tap or Flap,,,,ⱱ,,ɾ̪,,ɾ,,,...,,,NaN,NaN,,,,,NaN,NaN
Fricative,ɸ,β,f,v,θ,ð,s,z,ʃ,ʒ,...,ç,ʝ,x,ɣ,χ,ʁ,ħ,ʕ,h,ɦ
Lateral fricative,NaN,NaN,NaN,NaN,,,ɬ,ɮ,,,...,,,,,,,NaN,NaN,NaN,NaN
Approximant,,,,ʋ,,,,ɹ,,,...,,j,,ɰ,,,,,NaN,NaN
Lateral approximant,NaN,NaN,NaN,NaN,,l̪,,l,,,...,,ʎ,,ʟ,,,NaN,NaN,NaN,NaN


,0,1,2,3,4,5,6,7
Ejective Stop,pʼ,tʼ,ʈʼ,cʼ,kʼ,qʼ,ʡʼ,
Ejective Fricative,fʼ,θʼ,sʼ,ʃʼ,ʂʼ,ɕʼ,xʼ,χʼ
Ejective Affricate,tsʼ,t̠ʃʼ,ʈʂʼ,kxʼ,qχʼ,,,
Ejective Lateral,ɬʼ,tɬʼ,cʎ̝̊ʼ,kʟ̝̊ʼ,,,,
Click,kʘ,kǀ,kǃ,kǂ,kǁ,,,
Implosive Voiced,ɓ,ɗ,ᶑ,ʄ,ɠ,ʛ,,
Implosive Voiceless,ɓ̥,ɗ̥,ᶑ̥,ʄ̥,ɠ̊,ʛ̥,,


,Labial–alveolar,Labial–alveolar_v,Labial–retroflex,Labial–retroflex_v,Labial–palatal,Labial–palatal_v,Labial–velar,Labial–velar_v,Labial–uvular,Labial–uvular_v,Velarized alveolar,Velarized alveolar_v,Uvular–epiglottal,Uvular–epiglottal_v,Palatal-velar,Palatal-velar_v
Nasal,,n͡m,,ɳ͡m,,,,ŋ͡m,,,,,,,,
Plosive,t͡p,d͡b,ʈ͡p,ɖ͡b,,,k͡p,ɡ͡b,q͡p,,,,q͡ʡ,,,
Fricative/approximant,,,,,ɥ̊,ɥ,ʍ,w,,,,,,,ɧ,
Lateral approximant,,,,,,,,,,,,ɫ,,,,
Implosive,,,,,,,ɠ̊͜ɓ̥,ɠ͡ɓ,,,,,,,,
Ejective,,t͡pʼ,,,,,,,,,,,,,,


,Front unrounded,Front rounded,Near-front unrounded,Near-front rounded,Central unrounded,Central rounded,Near-back unrounded,Near-back rounded,Back unrounded,Back rounded
Close,i,y,,,ɨ,ʉ,,,ɯ,u
Near-close,,,ɪ,ʏ,,,,ʊ,,
Close-mid,e,ø,,,ɘ,ɵ,,,ɤ,o
Mid,e̞,ø̞,,,ə,,,,ɤ̞,o̞
Open-mid,ɛ,œ,,,ɜ,ɞ,,,ʌ,ɔ
Near-open,æ,,,ɐ,,,,,,
Open,a,ɶ,,,ä,,,,ɑ,ɒ


([8, 22], [7, 8], [6, 16], [7, 10])

In [133]:
# do we have duplicates?

table_phonemes_counts = (
    pd.concat(
        [
            cons_pl_tbl_df.stack(),
            cons_npl_tbl_df.stack(),
            cons_coart_tbl_df.stack(),
            vowels_tbl_df.stack(),
        ]
    )
    .replace("", np.nan)
    .dropna()
    .value_counts()
)

assert table_phonemes_counts[table_phonemes_counts == 1].all()

table_phonemes = table_phonemes_counts.index.to_series().reset_index(drop=True)

table_phonemes.head(3)

0      p
1    t͡p
2     ᶑ̥
dtype: object

In [134]:
# stack and split tables to seperate phonemes with/without specific diacritics

cons_pl_tbl_stacked_df = split_phoneme_series(
    cons_pl_tbl_df.stack().reset_index(drop=True).to_frame("Phoneme")
)
cons_npl_tbl_stacked_df = split_phoneme_series(
    cons_npl_tbl_df.stack().reset_index(drop=True).to_frame("Phoneme")
)
cons_coart_tbl_stacked_df = split_phoneme_series(
    cons_coart_tbl_df.stack().reset_index(drop=True).to_frame("Phoneme")
)
vowels_tbl_stacked_df = split_phoneme_series(
    vowels_tbl_df.stack().reset_index(drop=True).to_frame("Phoneme")
)

# combile all table phonemes

all_table_phonemes = (
    pd.concat(
        [
            cons_pl_tbl_stacked_df,
            cons_npl_tbl_stacked_df,
            cons_coart_tbl_stacked_df,
            vowels_tbl_stacked_df,
        ]
    )
)

table_phonemes_with_diacritics = (
    all_table_phonemes[all_table_phonemes.diacritic_count > 0]
    .Phoneme.replace("", np.nan)
    .dropna()
)
table_phonemes_without_diacritics = all_table_phonemes[
    all_table_phonemes.diacritic_count == 0
].Phoneme.replace("", np.nan).dropna()

display(table_phonemes_with_diacritics.head(3))
display(table_phonemes_without_diacritics.head(3))

4     t̪
5     d̪
25    n̪
Name: Phoneme, dtype: object

0    p
1    b
6    t
Name: Phoneme, dtype: object

For each langauge dialect, map all 4 phoneme tables against all the phonemes in the dialect.

If the table phoneme has a diacritic mark, compare the language phoneme and table phoneme directly. Return an array with one phoneme variant indicated for each matching table phoneme.

If the table phoneme doesn't have a diacritic mark, compare the base symbol of the language phoneme against the table phoneme. Return an array with all phoneme variants indicated for each matching table phoneme.

If the table cell is empty, return an array of zeros (length matching the number of phoneme variants) 

In [135]:
# combine duplicate phonemes that don't have explicit diacritic handling
# in one of the tables

handled_phonemes_df = dialect_phonemes_df[
    dialect_phonemes_df.Phoneme.isin(table_phonemes_with_diacritics.values)
]

print(handled_phonemes_df.shape)
display(handled_phonemes_df.head(3))

unhandled_phonemes_df = (
    dialect_phonemes_df[
        ~dialect_phonemes_df.Phoneme.isin(table_phonemes_with_diacritics.values)
    ][
        [
            "ISO6393",
            "LanguageName",
            "SpecificDialect",
            "symbol",
            "diacritics",
            "diacritic_count",
        ]
    ]
    .groupby(["ISO6393", "LanguageName", "SpecificDialect", "symbol"])
    .agg({"diacritics": "".join, "diacritic_count": "sum"})
    .reset_index()
)
unhandled_phonemes_df["Phoneme"] = unhandled_phonemes_df.symbol
unhandled_phonemes_df = unhandled_phonemes_df[
    [
        "ISO6393",
        "LanguageName",
        "SpecificDialect",
        "Phoneme",
        "symbol",
        "diacritics",
        "diacritic_count",
    ]
]


print(unhandled_phonemes_df.shape)
display(unhandled_phonemes_df.head(3))

phonemes_to_process_df = (
    pd.concat([handled_phonemes_df, unhandled_phonemes_df])
    .sort_values(["LanguageName", "SpecificDialect", "Phoneme"])
    .reset_index(drop=True)
)

display(phonemes_to_process_df.head(3))

(3669, 7)


,ISO6393,LanguageName,SpecificDialect,Phoneme,symbol,diacritics,diacritic_count
91,lbe,Lak,none,kʼ,k,ʼ,1
98,lbe,Lak,none,pʼ,p,ʼ,1
104,lbe,Lak,none,qʼ,q,ʼ,1


(80029, 7)


,ISO6393,LanguageName,SpecificDialect,Phoneme,symbol,diacritics,diacritic_count
0,aae,Arbëresh Albanian,Arbëresh Albanian (Hora e Arbëreshëvet),a,a,,0
1,aae,Arbëresh Albanian,Arbëresh Albanian (Hora e Arbëreshëvet),b,b,,0
2,aae,Arbëresh Albanian,Arbëresh Albanian (Hora e Arbëreshëvet),c,c,,0


,ISO6393,LanguageName,SpecificDialect,Phoneme,symbol,diacritics,diacritic_count
0,ktz,!XU,none,a,a,̃ː̃ːˤ̃ˤˤː̃ˤː,12
1,ktz,!XU,none,ae,ae,̞̞ˤ̞̃̃ˤ,7
2,ktz,!XU,none,ao,ao,̞̞ˤ̞̃̃ˤ,7


In [136]:
phonemes_to_process_by_dialect_df = (
    phonemes_to_process_df.groupby(["LanguageName", "SpecificDialect"])
)

In [168]:
def create_symbol_matrix(dialect_df):
    df = dialect_df.copy()
    df["cons_pl"] = np.where(
        dialect_df.Phoneme.isin(cons_pl_tbl_stacked_df.Phoneme.values),
        dialect_df.diacritics.map(
            lambda x: (valid_diacritics == x).to_numpy().astype(int)
        ).to_numpy(),
        dialect_df.Phoneme.map(
            lambda _: np.zeros(valid_diacritics.shape[0]),
        ).to_numpy(),
    )
    return df


cons_pl_processed = phonemes_to_process_by_dialect_df.apply(
    create_symbol_matrix, include_groups=False
)

cons_pl_processed[0:50]

ISO6393 Phoneme symbol    diacritics  \
LanguageName SpecificDialect                                           
!XU          none            0      ktz       a      a  ̃ː̃ːˤ̃ˤˤː̃ˤː   
                             1      ktz      ae     ae       ̞̞ˤ̞̃̃ˤ   
                             2      ktz      ao     ao       ̞̞ˤ̞̃̃ˤ   
                             3      ktz       b      b             ʼ   
                             4      ktz       d      d            ˠʼ   
                             5      ktz      dz     dz           ̤ˠʼ   
                             6      ktz      dʒ     dʒ        ̠̤̠ˠ̠ʼ   
                             7      ktz       e      e            ̞ː   
                             8      ktz      ei     ei          ̞̞̃̃   
                             9      ktz      eu     eu          ̞̞̃̃   
                             10     ktz      e̞      e             ̞   
                             11     ktz       i      i            ̃ː   
                             12     ktz      ia     ia                 
                             13     ktz       j      j                 
                             14     ktz       k      k             ʰ   
                             15     ktz     kǀ͓    kǀ͓             ʰ   
                             16     ktz    kǀ͓x   kǀ͓x                 
                             17     ktz      kǂ     kǂ             ʰ   
                             18     ktz     kǂx    kǂx                 
                             19     ktz     kǂ͓    kǂ͓           ˡˡʰ   
                             20     ktz    kǂ͓x   kǂ͓x             ˡ   
                             21     ktz      kǃ     kǃ             ʰ   
                             22     ktz     kǃx    kǃx                 
                             23     ktz      kʼ      k             ʼ   
                             24     ktz       m      m           ̤̰ː   
                             25     ktz       n      n                 
                             26     ktz       o      o  ̞̞̃ː̞ˤ̞̃ˤ̞ˤː   
                             27     ktz      oa     oa    ̞̞̞̃̃ˤ̞̃̃ˤ   
                             28     ktz      oe     oe            ̞̞   
                             29     ktz      oi     oi    ̞̞̞̃̃ˤ̞̃̃ˤ   
                             30     ktz      o̞      o             ̞   
                             31     ktz       p      p             ʰ   
                             32     ktz       s      s                 
                             33     ktz       t      t            ˠʰ   
                             34     ktz      ts     ts            ˠʰ   
                             35     ktz     tsʼ     ts             ʼ   
                             36     ktz      tʃ     tʃ         ̠̠ˠ̠ʰ   
                             37     ktz      tʼ      t             ʼ   
                             38     ktz    t̠ʃʼ     tʃ            ̠ʼ   
                             39     ktz       u      u          ̃ː̃ː   
                             40     ktz      ui     ui            ̃̃   
                             41     ktz       w      w                 
                             42     ktz       x      x                 
                             43     ktz       z      z                 
                             44     ktz       ŋ      ŋ             ˤ   
                             45     ktz     ŋǀ͓    ŋǀ͓         ̤̥ʰ̥ˀ   
                             46     ktz    ŋǀ͓x   ŋǀ͓x            ̥ˀ   
                             47     ktz      ŋǂ     ŋǂ         ̤̥ʰ̥ˀ   
                             48     ktz     ŋǂx    ŋǂx            ̥ˀ   
                             49     ktz     ŋǂ͓    ŋǂ͓     ˡ̤ˡ̥ˡʰ̥ˡˀ   

                                 diacritic_count  \
LanguageName SpecificDialect                       
!XU          none            0                12   
                             1                 7   
                             2                 7   
                           